<a href="https://colab.research.google.com/github/holictoweb/spark_deep_dive/blob/main/sparksql/pyspark_googledrive_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### colab + yfinance + google drive + pyspark

1. pyspark 설치 
2. yfinance를 통해 데이터 수집
3. google drive 상에 parquet 으로 데이터 저장
4. pyspark을 통해 해당 데이터를 테이블로 저장
5. 데이터 분석 진행

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit

spark = SparkSession.builder.appName('test_spark').getOrCreate()

In [66]:
spark.sql("create database IF NOT EXISTS stocklab")
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
| stocklab|
+---------+



In [119]:
# yfinance 를 통해 리스트 확보
import yfinance as yf

#data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30") #sample code 

ticker_list = ["005930.KS", ]

for ticker in ticker_list:
  pdf = yf.download(ticker, sdate='2020-01-01')

  '''
  df_schema = StructType([ \
    StructField("open"), DoubleType(), True), \
    StructField("high"), DoubleType(), True), \
    StructField("low"), DoubleType(), True), \
    StructField("close"), DoubleType(), True), \
    StructField("adfclose"), DoubleType(), True), \
    StructField("volume"), LongType(), True) \  
  ])
  '''


  df = spark.createDataFrame(pdf)  #df.show()
  
  #df.write.format('parquet').save('drive/MyDrive/data-warehouse/test')  # column Adj Close 와 관련한 이슈 발생
  # AnalysisException: Attribute name "Adj Close" contains invalid character(s) among " ,;{}()\n\t=". Please use alias to rename it. 

  ticker = ticker.split('.')[0]
  print(ticker)
  
  df = df.withColumnRenamed("Adj Close", "AdjClose").withColumn("Code", lit(ticker))

  path = 'drive/MyDrive/data-warehouse/stock_day'
  df.write.format('parquet').mode("overwrite").save(path)
  #df.show()
  create_table_sql = 'create table if not exists stocklab.stock_day using org.apache.spark.sql.parquet options( path  "'+ path +'")'
  spark.sql(create_table_sql)
  
  
  '''
  df.write.saveAsTable("stock_day_test")

  df_read = spark.read.format("parquet").load(path)
  df_read.show()
  '''

[*********************100%***********************]  1 of 1 completed
005930
+------+------+------+------+----------------+--------+------+
|  Open|  High|   Low| Close|        AdjClose|  Volume|  Code|
+------+------+------+------+----------------+--------+------+
|6000.0|6110.0|5660.0|6110.0|4761.18310546875|74195000|005930|
|5800.0|6060.0|5520.0|5580.0|4348.18408203125|74680000|005930|
|5750.0|5780.0|5580.0|5620.0|4379.35400390625|54390000|005930|
|5560.0|5670.0|5360.0|5540.0|     4317.015625|40305000|005930|
|5600.0|5770.0|5580.0|5770.0|4496.24072265625|46880000|005930|
|5820.0|6100.0|5770.0|5770.0|4496.24072265625|59745000|005930|
|5610.0|5740.0|5600.0|5720.0| 4457.2783203125|29220000|005930|
|5600.0|5740.0|5560.0|5710.0|4449.48583984375|41190000|005930|
|5720.0|5880.0|5680.0|5830.0|4542.99560546875|49375000|005930|
|6000.0|6180.0|5920.0|6100.0|4753.39111328125|63505000|005930|
|6160.0|6160.0|5980.0|6100.0|4753.39111328125|45260000|005930|
|6000.0|6040.0|5960.0|5960.0|4644.29736328

In [117]:
spark.sql ( "select * from stocklab.stock_day_test").show()
spark.sql("show tables").show()

+------+------+------+------+----------------+--------+------+
|  Open|  High|   Low| Close|        AdjClose|  Volume|  Code|
+------+------+------+------+----------------+--------+------+
|6000.0|6110.0|5660.0|6110.0|4761.18310546875|74195000|005930|
|5800.0|6060.0|5520.0|5580.0|4348.18408203125|74680000|005930|
|5750.0|5780.0|5580.0|5620.0|4379.35400390625|54390000|005930|
|5560.0|5670.0|5360.0|5540.0|     4317.015625|40305000|005930|
|5600.0|5770.0|5580.0|5770.0|4496.24072265625|46880000|005930|
|5820.0|6100.0|5770.0|5770.0|4496.24072265625|59745000|005930|
|5610.0|5740.0|5600.0|5720.0| 4457.2783203125|29220000|005930|
|5600.0|5740.0|5560.0|5710.0|4449.48583984375|41190000|005930|
|5720.0|5880.0|5680.0|5830.0|4542.99560546875|49375000|005930|
|6000.0|6180.0|5920.0|6100.0|4753.39111328125|63505000|005930|
|6160.0|6160.0|5980.0|6100.0|4753.39111328125|45260000|005930|
|6000.0|6040.0|5960.0|5960.0|4644.29736328125|40205000|005930|
|5860.0|6040.0|5820.0|6040.0|4706.63916015625|37315000|

In [ ]:
spark.sql("drop table stocklab.stock_day_test")

## pyspark 사용방법
- pip install pyspark 설치 이후 
  stored in directory 위치 확인 
  
/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

- spark home 지정

PYSPARK_PYTHON=python3 SPARK_HOME=~/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

In [1]:
!pip install pyspark


     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 39.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=2466b3861ca0b97f7248626980724d97eaff4ec66a2b511920cd0421a2cf9d3a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [6]:
!PYSPARK_PYTHON=python3 SPARK_HOME=~/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

In [2]:
!pip install yfinance
!pip install 

     |████████████████████████████████| 5.5MB 6.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=074b27b789cadcd7cd2b423d17dd62999efcd0ca3473ff3f67518c8bea34bb04
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [20]:
!ls drive/MyDrive/data-warehouse